In [1]:
import awkward as ak
import numpy as np
import uproot as uproot

In [2]:
filename = '/eos/user/g/glavizza/SWAN_projects/VBS_tagger/tree_Zjj_10k.root'
file = uproot.open(filename)

In [3]:
print(file.keys())

['GenParticles;2', 'GenParticles;1', 'LHEParticles;1', 'GenJets;1']


In [4]:
f_LHEParticles = file["LHEParticles;1"]
f_GenJets = file["GenJets;1"]
f_GenParticles = file["GenParticles;2"]

In [5]:
print(f_LHEParticles.keys())
print(f_GenJets.keys())
print(f_GenParticles.keys())

['LHEPart_pt', 'LHEPart_eta', 'LHEPart_phi', 'LHEPart_mass', 'LHEPart_pdgId', 'LHEPart_status']
['GenJet_pt', 'GenJet_eta', 'GenJet_phi', 'GenJet_mass', 'GenJet_nConstituents', 'GenJet_constituents']
['nGenPart', 'GenPart_pt', 'GenPart_eta', 'GenPart_phi', 'GenPart_mass', 'GenPart_pdgId']


In [6]:
LHEParticles = f_LHEParticles.arrays(f_LHEParticles.keys())
GenJets = f_GenJets.arrays(f_GenJets.keys())
GenParticles = f_GenParticles.arrays(f_GenParticles.keys())

In [7]:
# ------------------------ original function

def wrapInPhi(phi):
    return (phi + np.pi) % (2 * np.pi) - np.pi

def getJetConstituents(i, nConst, allConst):
    return allConst[sum(nConst[:i]):sum(nConst[:i+1])]


def event_jet_pull(GenJets, GenParticles, ev):
    genjets_pt = GenJets.GenJet_pt[ev]
    nConstituents = GenJets.GenJet_nConstituents[ev]
    constituents = GenJets.GenJet_constituents[ev]
    genParts_pt = GenParticles.GenPart_pt[ev]
    genParts_eta = GenParticles.GenPart_eta[ev]
    genParts_phi = GenParticles.GenPart_phi[ev]
    genJets_eta = GenJets.GenJet_eta[ev]
    genJets_phi = GenJets.GenJet_phi[ev]
        
    NJETS = len(GenJets.GenJet_pt[ev])
    genjets_pull = []
    
    pulls_m = []
    pulls_eta = []
    pulls_phi = []
    
    
    for i in range(NJETS):
        pull_m = 0
        pull_eta = 0
        pull_phi = 0
        
        genPartsInJetIndices = getJetConstituents(i, nConstituents, constituents)
        
        cs_pt  = genParts_pt[genPartsInJetIndices]
        cs_eta = genParts_eta[genPartsInJetIndices]
        cs_phi = genParts_phi[genPartsInJetIndices]
        
        jet_pt  = GenJets.GenJet_pt[ev][i]
        jet_eta = GenJets.GenJet_eta[ev][i]
        jet_phi = GenJets.GenJet_phi[ev][i]
        
        for c_pt, c_eta, c_phi in zip(cs_pt, cs_eta, cs_phi):
            pti = c_pt
            ri2 = (c_eta-jet_eta)**2+(c_phi-jet_phi)**2
            m = pti*ri2
            pull_m += m
            pull_eta += (c_eta-jet_eta)*m
            pull_phi += (c_phi-jet_phi)*m

        
        pull_m /= jet_pt
        pull_eta /= jet_pt
        pull_phi /= jet_pt
        
        pulls_m.append(pull_m)
        pulls_eta.append(pull_eta)
        pulls_phi.append(pull_phi)
        
        
    
    return pulls_m, pulls_eta, pulls_phi



In [8]:
# ------------------------ numba rewriting

import numpy as np
from numba import njit
import awkward.numba

@njit
def getJetConstituents_numba(i, nConst, allConst):
    start = 0
    for j in range(i):
        start += nConst[j]
    end = start + nConst[i]
    return allConst[start:end]

@njit
def wrapInPhi_numba(phi):
    x = (phi + np.pi) % (2 * np.pi) - np.pi
    return x

@njit
def event_jet_pull_numba(GenJets, GenParticles, ev):
    genjets_pt = GenJets.GenJet_pt[ev]
    nConstituents = GenJets.GenJet_nConstituents[ev]
    constituents = GenJets.GenJet_constituents[ev]
    genParts_pt = GenParticles.GenPart_pt[ev]
    genParts_eta = GenParticles.GenPart_eta[ev]
    genParts_phi = GenParticles.GenPart_phi[ev]
    genJets_eta = GenJets.GenJet_eta[ev]
    genJets_phi = GenJets.GenJet_phi[ev]
    
        
    NJETS = len(GenJets.GenJet_pt[ev])    
    pulls_m = []
    pulls_eta = []
    pulls_phi = []
    
    
    for i in range(NJETS):
        pull_m = 0
        pull_eta = 0
        pull_phi = 0
        
        genPartsInJetIndices = getJetConstituents_numba(i, nConstituents, constituents)

        cs_pt =[]
        cs_eta =[]
        cs_phi =[]
                
        for gpidx in genPartsInJetIndices:
            cs_pt.append(genParts_pt[gpidx])
            cs_eta.append(genParts_eta[gpidx])
            cs_phi.append(genParts_phi[gpidx])
        
        # print ('cs_pt', cs_pt)
        
        jet_pt  = GenJets.GenJet_pt[ev][i]
        jet_eta = GenJets.GenJet_eta[ev][i]
        jet_phi = GenJets.GenJet_phi[ev][i]
        
        jet_phi = wrapInPhi_numba(jet_phi)        
        
        for c_pt, c_eta, c_phi in zip(cs_pt, cs_eta, cs_phi):
            c_phi = wrapInPhi_numba(c_phi)
            pti = c_pt
            ri2 = (c_eta-jet_eta)**2+(c_phi-jet_phi)**2
            m = pti*ri2
            pull_m += m
            pull_eta += (c_eta-jet_eta)*m
            pull_phi += (c_phi-jet_phi)*m

        
        pull_m /= jet_pt
        pull_eta /= jet_pt
        pull_phi /= jet_pt
        
        pulls_m.append(pull_m)
        pulls_eta.append(pull_eta)
        pulls_phi.append(pull_phi)
        
        
    
    return pulls_m, pulls_eta, pulls_phi

In [10]:
import time
start = time.time()
for i in range(1000):
    #event_jet_pull(GenJets, GenParticles, i) # 95.97 s on 1k files
    event_jet_pull_numba(GenJets, GenParticles, i) # 2.83 s on 1k files, 3.88 s on 10k events (all)

print (time.time() - start)

2.8369152545928955
